# ConvLSTM

In [10]:
import numpy as np

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
# import keras
# from keras.models import Sequential
# from keras.layers import ConvLSTM2D, Dense, InputLayer, BatchNormalization, Permute

Let's just make a model and experiment with it

In [2]:
frames = 24
channels = 5
pixels_x = 21
pixels_y = 21

In [3]:
model = tf.keras.Sequential(name='convlstm_model_0')
model.add(layers.InputLayer(input_shape=(frames, channels, pixels_x, pixels_y)))
model.add(layers.ConvLSTM2D(
    filters=20, kernel_size=(5,5), padding='same', data_format='channels_first',return_sequences=True))
model.add(layers.BatchNormalization(axis=1, ))
model.add(layers.ConvLSTM2D(
    filters=10, kernel_size=(3,3), padding='same', data_format='channels_first',return_sequences=True))
model.add(layers.BatchNormalization(axis=1, ))
model.add(layers.ConvLSTM2D(
    filters=5, kernel_size=(1,1), padding='same', data_format='channels_first',return_sequences=True))
model.add(layers.BatchNormalization(axis=1, ))

model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['mean_absolute_error'])

In [4]:
model.summary()

Model: "convlstm_model_0"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 24, 20, 21, 21)    50080     
_________________________________________________________________
batch_normalization (BatchNo (None, 24, 20, 21, 21)    96        
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 24, 10, 21, 21)    10840     
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 10, 21, 21)    96        
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 24, 5, 21, 21)     320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 5, 21, 21)     96        
Total params: 61,528
Trainable params: 61,384
Non-trainable params: 144
____________________________________________

## try training

In [6]:
import glob
import xarray as xr
def generate_arrays(img_dir, slice_size=24):
    """
    A generator that returns one 24-hour slice as input, and the subsequent 24-hour slice as output
    """
    # get list of netcdf files in img_dir
    netcdf_dirs = sorted(glob.glob(img_dir+"/*.nc"))
    file_index = 0
    # open first netcdf file
    ds = xr.open_dataset(netcdf_dirs[file_index])
    # counter is for hourly time slices. months with 31 days have 744 hours
    counter = 0
    while True: # generator needs to run infinitely
        
        # get input slice
        input_images = ds.isel( time=slice(counter, counter + slice_size)).to_array().values
        
        # check if we're at the end of the month
        if counter+2*slice_size > ds.sizes['time']:
            # reset slice counter, increment to next netcdf file, open it, get output images
            counter = 0
            file_index += 1
            if file_index == len(netcdf_dirs):
                raise EOFError("End of the line. Last file at ", netcdf_dirs[file_index-1])
                
            ds = xr.open_dataset(netcdf_dirs[file_index])
            # take slice 0-24 as output-image
            output_images = ds.isel( time=slice(counter, counter + slice_size)).to_array().values
            # set counter to -slice_size to reset for input on next iteration
            counter -= slice_size
        # get output slice right after input slice
        else:
            output_images = ds.isel( time=slice(counter+slice_size, counter + 2*slice_size)).to_array().values
        
        # switch frames and channel axes
        input_images = np.moveaxis(input_images, 0, 1)
        output_images = np.moveaxis(output_images, 0, 1)
        # reshape values
        input_images = input_images.reshape(-1, frames, channels, pixels_x, pixels_y)
        output_images = output_images.reshape(-1, frames, channels, pixels_x, pixels_y)
        yield (input_images, output_images)
        counter += slice_size

In [7]:
train_file_path = "../data/train"
validate_file_path = "../data/validate"

In [11]:
history = model.fit(
    generate_arrays(train_file_path),
    steps_per_epoch = 365*3 - 1,
    epochs = 20,
    verbose = 1,
    shuffle = False,
    initial_epoch = 0,
    validation_steps = 365 - 1,
    validation_data = generate_arrays(validate_file_path),
    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1094 steps, validate for 364 steps
Epoch 1/20
 940/1094 [========================>.....] - ETA: 5:28 - loss: 1816039.0779 - mean_absolute_error: 19772.2676

KeyboardInterrupt: 

## Save model to ```../models/```

In [ ]:
import pickle
import datetime
current_datetime = datetime.datetime.now().strftime("%Y_%M_%d_%H%M")
pickle.dump( history, open( "../models/"+current_datetime+"_convlstm.pkl", "wb" ) )

In [ ]:
# #rewrite model in Functional style

# inp = InputLayer(input_shape=(channels, frames, pixels_x, pixels_y))
# permuted = Permute((2,1,3,4))(inp)
# convLstm1 = ConvLSTM2D(filters=20, kernel_size=(5,5), padding='same', 
#                        data_format='channels_first',return_sequences=True)(permuted)
# batch_norm1 = BatchNormalization(axis=1, )(convLstm1)
# convLstm2 = ConvLSTM2D(filters=10, kernel_size=(5,5), padding='same', 
#                        data_format='channels_first',return_sequences=True)(batch_norm1)
# batch_norm2 = BatchNormalization(axis=1, )(convLstm2)
# conv_output = ConvLSTM2D(filters=10, kernel_size=(1,1), padding='same', 
#                        data_format='channels_first',return_sequences=True)(batch_norm2)

# model = Model(inputs = [inp], outputs = [conv_output])
# model.compile(loss='sparse_categorical_crossentropy',
#                   optimizer='adadelta',
#                   metrics=['accuracy'])